In [1]:
from __future__ import print_function, division
from bs4 import BeautifulSoup
import time, sys
from IPython.display import clear_output
import requests
import pandas as pd

requests.__path__

['/home/danish/anaconda3/envs/metis/lib/python3.6/site-packages/requests']

All of the Spotify chart URLs are in the same format. The only thing that changes is the current date that it is being pulled from. First let's generate a list with all of the dates that we want to look at.

In [52]:
url_base = "https://spotifycharts.com/regional/global/daily/"
timelist = pd.date_range(pd.datetime.today()-pd.Timedelta(days=366), periods=365).to_list()
urls = []

for i in range(0,len(timelist)):
    urls.append(url_base + str(timelist[i]).split()[0])

Next we will go ahead and pull all of the dates that we're looking at from the website. I built in a 3 second delay which does cause the program to take a little time, especially when scraping a year's worth of data. However this should be enough to prevent us from being blacklisted.

In [53]:
soup = []

for i in range(0,len(urls)):
    response = requests.get(urls[i])

    print("Page number {}, response code is: ".format(i+1) + str(response.status_code))

    page = response.text

    soup.append(BeautifulSoup(page, "lxml"))
    
    time.sleep(3)

Page number 1, response code is: 200
Page number 2, response code is: 200
Page number 3, response code is: 200
Page number 4, response code is: 200
Page number 5, response code is: 200
Page number 6, response code is: 200
Page number 7, response code is: 200
Page number 8, response code is: 200
Page number 9, response code is: 200
Page number 10, response code is: 200
Page number 11, response code is: 200
Page number 12, response code is: 200
Page number 13, response code is: 200
Page number 14, response code is: 200
Page number 15, response code is: 200
Page number 16, response code is: 200
Page number 17, response code is: 200
Page number 18, response code is: 200
Page number 19, response code is: 200
Page number 20, response code is: 200
Page number 21, response code is: 200
Page number 22, response code is: 200
Page number 23, response code is: 200
Page number 24, response code is: 200
Page number 25, response code is: 200
Page number 26, response code is: 200
Page number 27, respo

Page number 214, response code is: 200
Page number 215, response code is: 200
Page number 216, response code is: 200
Page number 217, response code is: 200
Page number 218, response code is: 200
Page number 219, response code is: 200
Page number 220, response code is: 200
Page number 221, response code is: 200
Page number 222, response code is: 200
Page number 223, response code is: 200
Page number 224, response code is: 200
Page number 225, response code is: 200
Page number 226, response code is: 200
Page number 227, response code is: 200
Page number 228, response code is: 200
Page number 229, response code is: 200
Page number 230, response code is: 200
Page number 231, response code is: 200
Page number 232, response code is: 200
Page number 233, response code is: 200
Page number 234, response code is: 200
Page number 235, response code is: 200
Page number 236, response code is: 200
Page number 237, response code is: 200
Page number 238, response code is: 200
Page number 239, response

Once the data has been pulled, we need to obtain the tracks, artists, stream count, and the date. In addition, we'll also collect the song urls as they are necessary to pull some additional information from the API in a later step.

In [54]:
def chart_data_pull(site_data, index):
    song_titles_html = site_data[index].find_all(class_='chart-table-track')[1:]
    streams_html = site_data[index].find_all(class_='chart-table-streams')[1:]
    song_urls_html = site_data[index].find_all(class_='chart-table-image')
    
    for i in range(0,len(song_titles_html)):
        songs.append(song_titles_html[i].text.split('by')[0].strip('\n').strip())
        song_urls.append(song_urls_html[i].find('a', href=True)['href'])
        artists.append(song_titles_html[i].text.split('by')[1].strip('\n').strip())
        streams.append(streams_html[i].text.replace(',',''))
        date.append(str(timelist[index]).split()[0])

In [55]:
songs = []
song_urls = []
artists = []
streams = []
date = []

In [56]:
for i in range(0,len(soup)):
    chart_data_pull(soup, i)

Finally, we will put the data together and export it to a csv file for future work.

In [57]:
song_data = list(zip(songs, song_urls, artists, streams, date))

columns = ['Title','Song Links', 'Artist', 'Streams', 'Date']
df = pd.DataFrame(song_data, columns=columns)

In [58]:
df.sample(10)

,Title,Song Links,Artist,Streams,Date
48388,"I Can’t Get Enough (benny blanco, Selena Gomez...",https://open.spotify.com/track/1WSGTYKLEpKfNQY...,benny blanco,655186,2019-06-01
10657,I Like It,https://open.spotify.com/track/58q2HKrzhC3ozto...,Cardi B,1105530,2018-11-25
34426,Pa Mí - Remix,https://open.spotify.com/track/224EVTFiLsdQign...,Dalex,1724309,2019-03-24
38900,listen before i go,https://open.spotify.com/track/0tMSssfxAL2oV8V...,Billie Eilish,946837,2019-04-15
63478,Hot (feat. Gunna),https://open.spotify.com/track/5Z8HZM6iQMhhqyP...,Young Thug,1058520,2019-08-16
4429,Fine China,https://open.spotify.com/track/5274I4mUMnYczye...,Future,1766884,2018-10-25
60607,Ransom,https://open.spotify.com/track/1kBbgxAUc4HQv7c...,Lil Tecca,3458991,2019-08-02
24486,Undecided,https://open.spotify.com/track/7uiBocndm12aKbs...,Chris Brown,1014508,2019-02-02
21572,Love Lies (with Normani),https://open.spotify.com/track/45Egmo7icyopuzJ...,Khalid,753869,2019-01-18
27635,imagine,https://open.spotify.com/track/39LmTF9RgyakzSY...,Ariana Grande,1686179,2019-02-18


In [59]:
df.describe()

,Title,Song Links,Artist,Streams,Date
count,73000,73000,73000,73000,73000
unique,1623,1891,523,70589,365
top,Happier,https://open.spotify.com/track/0pqnGHJpmpxLKif...,Billie Eilish,712319,2019-03-21
freq,493,365,3010,4,200


In [60]:
df.to_csv(path_or_buf='./chart_data.csv')